In [1]:
import pandas as pd
from sklearn.metrics import classification_report
import pickle

In [2]:
# 1. Load and preprocess the second‐order dataset
df2 = pd.read_csv("/kaggle/input/ethereum-transactions-for-fraud-detection/second_order_df.csv").head(1000)
df2.columns = df2.columns.str.strip()

# Drop any stray index column
if 'Unnamed: 0' in df2.columns:
    df2.drop('Unnamed: 0', axis=1, inplace=True)

# Convert Unix timestamp and extract time features
df2['TimeStamp'] = pd.to_datetime(df2['TimeStamp'], unit='s')
df2['hour']  = df2['TimeStamp'].dt.hour
df2['day']   = df2['TimeStamp'].dt.day
df2['month'] = df2['TimeStamp'].dt.month
df2['year']  = df2['TimeStamp'].dt.year

In [3]:
# 2. Prepare features & target
features = ['BlockHeight', 'Value', 'hour', 'day', 'month', 'year', 'From', 'To']
X2 = df2[features]
y2 = df2['isError']

In [4]:
# 3. Load all your saved pipelines
model_keys = ["RF", "MLP", "XGB", "RF+MLP", "RF+XGB", "MLP+XGB", "RF+MLP+XGB"]
pipelines = {}
for key in model_keys:
    fname = f"/kaggle/input/comp517_models/scikitlearn/default/1/pipeline_{key.replace('+','_')}.pkl"
    with open(fname, "rb") as f:
        pipelines[key] = pickle.load(f)

In [5]:
# 4. Evaluate each model on the second‐order set and summarize correctness
summary = []
total = len(y2)

for name, pipe in pipelines.items():
    y_pred = pipe.predict(X2)

    # Counts
    valid_count = int((y_pred == 0).sum())
    error_count = int((y_pred == 1).sum())
    correct    = int((y_pred == y2).sum())
    accuracy   = correct / total

    # Correctness within predicted-valid
    correct_valid = int(((y_pred == 0) & (y2 == 0)).sum())
    valid_correct_pct = correct_valid / valid_count if valid_count > 0 else 0.0

    # Correctness within predicted-error
    correct_error = int(((y_pred == 1) & (y2 == 1)).sum())
    error_correct_pct = correct_error / error_count if error_count > 0 else 0.0

    summary.append({
        "Model":            name,
        "Valid (0)":        valid_count,
        "Error (1)":        error_count,
        "Correct":          correct,
        "Total":            total,
        "Accuracy":         round(accuracy, 6),
        "Valid_Correct_%":  round(valid_correct_pct, 6),
        "Error_Correct_%":  round(error_correct_pct, 6)
    })

summary_df = pd.DataFrame(summary)
print("Overall Summary with Class-wise Correctness %:")
print(summary_df)

Overall Summary with Class-wise Correctness %:
        Model  Valid (0)  Error (1)  Correct  Total  Accuracy  \
0          RF        996          4     1000   1000     1.000   
1         MLP       1000          0      996   1000     0.996   
2         XGB       1000          0      996   1000     0.996   
3      RF+MLP        998          2      998   1000     0.998   
4      RF+XGB       1000          0      996   1000     0.996   
5     MLP+XGB       1000          0      996   1000     0.996   
6  RF+MLP+XGB       1000          0      996   1000     0.996   

   Valid_Correct_%  Error_Correct_%  
0         1.000000              1.0  
1         0.996000              0.0  
2         0.996000              0.0  
3         0.997996              1.0  
4         0.996000              0.0  
5         0.996000              0.0  
6         0.996000              0.0  
